# Additional information for ADV Python
the information in here supports different questions asked by the students


## what is a use case for class decorator
Decorators are a useful addition to Python, allowing a programmer the ability to modify the behaviour of function or class.  

They give the facitilty to wrap another function in order to extend the behaviour of the wrapped function, without permanently modifying it.   

We can define a decorator as a class and to do that we have to use a `__call__` method. When a user needs to create an object that acts as a function then function decorator needs to return an object that acts like a function, so __call__ can be useful.   

This can be used with logging or some other administrational requirement of the code, profiling is an example:


In [6]:
#Use a decorator to profile code

from time import time
class ProfileTimer:

	def __init__(self, func):
		self.function = func

	def __call__(self, *args, **kwargs):
    	start_time = time()
    	result = self.function(*args, **kwargs)
    	end_time = time()
        time_in_execution = end_time - start_time
        print(f"Execution took {time_in_execution} seconds"})
        return result


# use the class as a decorator
@ProfileTimer
def some_function(delay):
	from time import sleep
	sleep(delay)

some_function(3)


TabError: inconsistent use of tabs and spaces in indentation (<string>, line 10)

Can we make something that performs data validation. The task is to develop a Python program that creates a data validation library utilizing Python's "dataclasses" and type hints. This library enables automatic validation of "dataclass" fields based on specified type hints and custom validation functions. By integrating these features, the program will facilitate robust data validation, ensuring data conforms to expected types and constraints. This will enhance data integrity and reducing runtime errors in applications.

In [1]:
# Import necessary modules
from dataclasses import dataclass, field, fields, Field
from typing import Any, Callable, List, get_type_hints

# Custom exception for validation errors
class ValidationError(Exception):
    pass

# Function to validate dataclass fields
def validate(instance):
    cls = instance.__class__
    hints = get_type_hints(cls)
    # Iterate through fields and type hints
    for name, type_hint in hints.items():
        value = getattr(instance, name)
        # Check if value matches type hint
        if not isinstance(value, type_hint):
            raise ValidationError(f"Field '{name}' expects {type_hint}, got {type(value)}")
        # Check for custom validators
        field: Field = next(f for f in fields(cls) if f.name == name)
        if 'validators' in field.metadata:
            for validator in field.metadata['validators']:
                validator(instance, value)

# Decorator to define a validator function
def validator(func: Callable[[Any, Any], None]):
    if not callable(func):
        raise ValueError("Validator must be callable")
    func._is_validator = True
    return func

# Function to define a field with validation
def field_with_validation(*, default: Any = None, default_factory: Callable[[], Any] = None, validators: List[Callable[[Any, Any], None]] = None) -> Field:
    # Ensure only one of default or default_factory is provided
    if default is not None and default_factory is not None:
        raise ValueError('cannot specify both default and default_factory')
    
    metadata = {}
    # Attach validators to metadata
    if validators:
        metadata['validators'] = validators
    
    # Define field with appropriate parameters
    if default is not None:
        return field(default=default, metadata=metadata)
    elif default_factory is not None:
        return field(default_factory=default_factory, metadata=metadata)
    else:
        return field(metadata=metadata)

# Dataclass representing a User with validation
@dataclass
class User:
    # Define fields with validation
    name: str
    age: int = field_with_validation(default=0, validators=[
        validator(lambda instance, value: value >= 0 or ValidationError("Age must be non-negative")),
        validator(lambda instance, value: value <= 150 or ValidationError("Age must be 150 or less"))
    ])
    email: str = field_with_validation(default="", validators=[
        validator(lambda instance, value: "@" in value or ValidationError("Invalid email address"))
    ])

# Example usage
try:
    user = User(name="Arsen", age=30, email="arsen@example.com")
    validate(user)
    print("User is valid")
except ValidationError as e:
    print(f"Validation error: {e}")


User is valid
